# Solr Data Preprocessing

## 1. Synonyms
In this section, we will get the synonyms of the terms in the comments. First we will need to import the necessary libraries and download the WordNet dataset.

First, we need to define that we will use synonyms

In [ ]:
import nltk
import pandas as pd
import requests
import json

In [ ]:
url = "http://localhost:8983/solr/comments/schema/analysis/synonyms/en"

payload = json.dumps({
    "class": "org.apache.solr.rest.schema.analysis.ManagedSynonymGraphFilterFactory$SynonymManager"
})
headers = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

In [ ]:
url = "http://localhost:8983/solr/submissions/schema/analysis/synonyms/en"

payload = json.dumps({
    "class": "org.apache.solr.rest.schema.analysis.ManagedSynonymGraphFilterFactory$SynonymManager"
})
headers = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

In [ ]:
nltk.download('wordnet')

After, setting up the required libraries, we can continue with getting the unique terms from the comments in the DataFrame.

In [ ]:
# Define the set to hold the words
comment_terms = set()
submission_terms = set()

# Read the comments from the csv
comments = pd.read_csv('./subreddits/all/comments_cleaned_50.csv')

# Read the submissions from the csv
submissions = pd.read_csv('./subreddits/all/submissions.csv')

# Iterate over the comments
for index, row in comments.iterrows():
    # Split the comment into words
    words = row['body'].split()
    # Add each word to the set of terms
    for w in words:
        comment_terms.add(str.lower(w))

# Iterate over the submissions
for index, row in submissions.iterrows():
    words = []

    # Check if 'selftext' is not NaN and then split into words
    if pd.notna(row['selftext']):
        words.extend(row['selftext'].split())

    # Check if 'title' is not NaN and then split into words
    if pd.notna(row['title']):
        words.extend(row['title'].split())
    # Add each word to the set of terms
    for w in words:
        submission_terms.add(str.lower(w))

In [ ]:
# Get the synonyms of the words
from nltk.corpus import wordnet

comments_synonyms = {}
submissions_synonyms = {}

for word in comment_terms:
    comments_synonyms[word] = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            comments_synonyms[word].append(lemma.name())

for word in submission_terms:
    submissions_synonyms[word] = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            submissions_synonyms[word].append(lemma.name())

# remove the ones with empty list values
comments_synonyms = {k: v for k, v in comments_synonyms.items() if v}

# remove the ones with empty list values
submissions_synonyms = {k: v for k, v in submissions_synonyms.items() if v}


In [ ]:
# Save the synonyms as a JSON file
with open('comments_synonyms.json', 'w') as f:
    json.dump(comments_synonyms, f)
with open('submissions_synonyms.json', 'w') as f:
    json.dump(submissions_synonyms, f)

In [ ]:
# Read the synonyms from the JSON file
with open('comments_synonyms.json') as f:
    comments_synonyms = json.load(f)
with open('submissions_synonyms.json') as f:
    submissions_synonyms = json.load(f)

# Send request to the Solr server to add the synonyms
comments_url = "http://localhost:8983/solr/comments/schema/analysis/synonyms/en"
submissions_url = "http://localhost:8983/solr/submissions/schema/analysis/synonyms/en"
comments_payload = json.dumps(comments_synonyms)
submissions_payload = json.dumps(submissions_synonyms)
headers = {'Content-type': 'application/json'}
c_response = requests.request("PUT", comments_url, headers=headers, data=comments_payload)
s_response = requests.request("PUT", submissions_url, headers=headers, data=submissions_payload)
print(c_response.text)
print(s_response.text)

Now, let's create the necessary fields

In [ ]:
url = 'http://localhost:8983/solr/submissions/schema'
headers = {'Content-type': 'application/json'}
data = {
    "add-field": {
        "name": "title",
        "type": "sgm_text_en",
        "stored": True,
        "indexed": True,
        "multiValued": False
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.text)

In [ ]:
url = 'http://localhost:8983/solr/comments/schema'
headers = {'Content-type': 'application/json'}
data = {
    "add-field": {
        "name": "body",
        "type": "sgm_text_en",
        "stored": True,
        "indexed": True,
        "multiValued": False
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.text)

Now, we will upload the files

In [ ]:
# URL of the Solr update endpoint
url = 'http://localhost:8983/solr/submissions/update'

# Headers to specify that the payload is XML
headers = {'Content-Type': 'text/csv'}

# Load the XML data from a file
with open('subreddits/all/submissions_cleaned_50.csv', 'rb') as file:
    data = file.read()

# Send the POST request with the XML data
response = requests.post(url, headers=headers, data=data)

# Print the HTTP response text
print(response.text)

In [ ]:
# URL of the Solr update endpoint
url = 'http://localhost:8983/solr/comments/update'

# Headers to specify that the payload is XML
headers = {'Content-Type': 'text/csv'}

# Load the XML data from a file
with open('./subreddits/all/comments_cleaned_50_class.csv', 'rb') as file:
    data = file.read()

# Send the POST request with the XML data
response = requests.post(url, headers=headers, data=data)

# Print the HTTP response text
print(response.text)